<a href="https://colab.research.google.com/github/sberniz/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/module4-Deploy/Santiago_Berniz_LS_DS_434_Deploy_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 4*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Apply regularization techniques to your model. 

*Don't forgot to switch to GPU on Colab!*

In [2]:
# Imports
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import ReLU
import tensorflow as tf
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:

def load_quickdraw10(path):
  data = np.load(path)
  X = data['arr_0']
  y = data['arr_1']
  X = X.astype('float32') / 255.

  return train_test_split(X,y,test_size=0.2, train_size=0.8, random_state=42)


X_train, X_test, y_train, y_test = load_quickdraw10('quickdraw10.npz')
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((80000, 784), (80000,), (20000, 784), (20000,))

In [4]:
X_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.1882353 , 0.42745098, 0.42745098, 0.27450982,
       0.04705882, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.24313726, 0.7882353 , 1.        ,
       1.        , 1.        , 1.        , 0.8627451 , 0.17647

## Regularization

Using your best performing model from the previous module, apply each of the following regularization strategies: 
* Early Stopping
* Dropout
* Weight Decay
* Weight Constraint


In [5]:
#Early Stopping 
logdir = os.path.join("logs","EarlyStopping-loss")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

In [6]:
#Train Model with callback
model = tf.keras.Sequential([
  Dense(128, input_dim=784),
  ReLU(negative_slope=.01),
  Dense(128),
  ReLU(negative_slope=0.01),
  Dense(128),
  ReLU(negative_slope=.01),
  Dense(10,activation='softmax')
])
#Compile Model 
model.compile(loss='sparse_categorical_crossentropy',optimizer='nadam',metrics=['accuracy'])
#Fit model
model.fit(X_train,y_train, epochs=99, validation_data=(X_test,y_test), callbacks=[tensorboard_callback,stop])

Epoch 1/99
   1/2500 [..............................] - ETA: 0s - loss: 2.3790 - accuracy: 0.0625WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
2500/2500 [==============================] - 10s 4ms/step - loss: 0.6312 - accuracy: 0.8083 - val_loss: 0.4924 - val_accuracy: 0.8485
Epoch 2/99
2500/2500 [==============================] - 10s 4ms/step - loss: 0.4490 - accuracy: 0.8630 - val_loss: 0.4472 - val_accuracy: 0.8625
Epoch 3/99
2500/2500 [==============================] - 10s 4ms/step - loss: 0.3871 - accuracy: 0.8801 - val_loss: 0.4256 - val_accuracy: 0.8748
Epoch 4/99
2500/2500 [==============================] - 9s 4ms/step - loss: 0.3445 - accuracy: 0.8933 - val_loss: 0.4264 - val_accuracy: 0.8730
Epoch 5/99
2500/2500 [==============================] - 9

In [7]:
model.evaluate(X_test,y_test)

625/625 [==============================] - 1s 2ms/step - loss: 0.4537 - accuracy: 0.8736


[0.4536511301994324, 0.8736000061035156]

In [8]:
#Dropout Only
from tensorflow.keras.layers import Dropout

logdir = os.path.join("logs", "EarlyStopping+WeightConstraint+Dropout")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)

model_w_dropout = tf.keras.Sequential([
    Dense(512, input_dim=784),
    ReLU(negative_slope=.01),
    Dropout(.2),
    Dense(512),
    ReLU(negative_slope=.01),
    Dropout(.2),
    Dense(512),
    ReLU(negative_slope=.01),
    Dropout(.2),
    Dense(10, activation='softmax')
])

model_w_dropout.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

model_w_dropout.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/99
2500/2500 [==============================] - 11s 4ms/step - loss: 0.6487 - accuracy: 0.8019 - val_loss: 0.4850 - val_accuracy: 0.8495
Epoch 2/99
2500/2500 [==============================] - 10s 4ms/step - loss: 0.4845 - accuracy: 0.8543 - val_loss: 0.4386 - val_accuracy: 0.8675
Epoch 3/99
2500/2500 [==============================] - 11s 4ms/step - loss: 0.4305 - accuracy: 0.8679 - val_loss: 0.4346 - val_accuracy: 0.8672
Epoch 4/99
2500/2500 [==============================] - 10s 4ms/step - loss: 0.3942 - accuracy: 0.8813 - val_loss: 0.4262 - val_accuracy: 0.8784
Epoch 5/99
2500/2500 [==============================] - 10s 4ms/step - loss: 0.3624 - accuracy: 0.8905 - val_loss: 0.4246 - val_accuracy: 0.8770
Epoch 6/99
2500/2500 [==============================] - 10s 4ms/step - loss: 0.3391 - accuracy: 0.8967 - val_loss: 0.4237 - val_accuracy: 0.8778
Epoch 7/99
2500/2500 [==============================] - 10s 4ms/step - loss: 0.3198 - accuracy: 0.9027 - val_loss: 0.4213 - val_ac

In [9]:
model_w_dropout.evaluate(X_test,y_test)

625/625 [==============================] - 1s 2ms/step - loss: 0.4429 - accuracy: 0.8798


[0.4428623914718628, 0.8798499703407288]

In [10]:
#Decay Only
from tensorflow.keras import regularizers

logdir = os.path.join("logs", "EarlyStopping+L2_WeightDecay")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

model_w_decay = tf.keras.Sequential([
    Dense(512, input_dim=784, kernel_regularizer=regularizers.l2(0.01)),
    ReLU(negative_slope=.01),
    Dense(512, kernel_regularizer=regularizers.l2(0.01)),
    ReLU(negative_slope=.01),
    Dense(512, kernel_regularizer=regularizers.l2(0.01)),
    ReLU(negative_slope=.01),
    Dense(10, activation='softmax')
])

model_w_decay.compile(loss='sparse_categorical_crossentropy', optimizer='nadam',
              metrics=['accuracy'])

model_w_decay.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])


Epoch 1/99
2500/2500 [==============================] - 11s 4ms/step - loss: 1.4320 - accuracy: 0.7564 - val_loss: 0.9424 - val_accuracy: 0.8007
Epoch 2/99
2500/2500 [==============================] - 11s 4ms/step - loss: 0.9016 - accuracy: 0.8047 - val_loss: 0.8551 - val_accuracy: 0.8126
Epoch 3/99
2500/2500 [==============================] - 11s 4ms/step - loss: 0.8430 - accuracy: 0.8169 - val_loss: 0.8123 - val_accuracy: 0.8218
Epoch 4/99
2500/2500 [==============================] - 11s 4ms/step - loss: 0.8177 - accuracy: 0.8206 - val_loss: 0.8094 - val_accuracy: 0.8271
Epoch 5/99
2500/2500 [==============================] - 11s 4ms/step - loss: 0.8016 - accuracy: 0.8230 - val_loss: 0.7741 - val_accuracy: 0.8254
Epoch 6/99
2500/2500 [==============================] - 11s 4ms/step - loss: 0.7960 - accuracy: 0.8229 - val_loss: 0.7811 - val_accuracy: 0.8224
Epoch 7/99
2500/2500 [==============================] - 11s 4ms/step - loss: 0.7795 - accuracy: 0.8242 - val_loss: 0.7900 - val_ac

In [11]:
model_w_decay.evaluate(X_test,y_test)

625/625 [==============================] - 1s 2ms/step - loss: 0.7904 - accuracy: 0.8177


[0.7903968095779419, 0.8177499771118164]

In [12]:
#Constrain Only with early stop 
from tensorflow.keras.constraints import MaxNorm

weight_constrain = MaxNorm(max_value=2)

logdir = os.path.join("logs", "EarlyStopping+WeightConstraint")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

model_w_constrain = tf.keras.Sequential([
    Dense(512, input_dim=784, kernel_constraint=weight_constrain),
    ReLU(negative_slope=.01),
    Dense(512, kernel_constraint=weight_constrain),
    ReLU(negative_slope=.01),
    Dense(512, kernel_constraint=weight_constrain),
    ReLU(negative_slope=.01),
    Dense(10, activation='softmax')
])

model_w_constrain.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

model_w_constrain.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/99
2500/2500 [==============================] - 11s 4ms/step - loss: 0.5827 - accuracy: 0.8212 - val_loss: 0.4857 - val_accuracy: 0.8494
Epoch 2/99
2500/2500 [==============================] - 10s 4ms/step - loss: 0.4150 - accuracy: 0.8737 - val_loss: 0.4267 - val_accuracy: 0.8695
Epoch 3/99
2500/2500 [==============================] - 10s 4ms/step - loss: 0.3531 - accuracy: 0.8908 - val_loss: 0.4396 - val_accuracy: 0.8708
Epoch 4/99
2500/2500 [==============================] - 10s 4ms/step - loss: 0.3188 - accuracy: 0.9003 - val_loss: 0.4179 - val_accuracy: 0.8792
Epoch 5/99
2500/2500 [==============================] - 10s 4ms/step - loss: 0.2954 - accuracy: 0.9068 - val_loss: 0.4309 - val_accuracy: 0.8752


In [17]:
model_w_constrain.evaluate(X_test,y_test)

625/625 [==============================] - 1s 2ms/step - loss: 0.4309 - accuracy: 0.8752


[0.43086132407188416, 0.875249981880188]

## Deploy

Save your model's weights using the Checkpoint function. Try reloading the model and making inference on your validation dataset.

In [22]:
#Train Model 
import tensorflow as tf
#Save weights
cpoint = tf.keras.callbacks.ModelCheckpoint("weights_best.h5",
                                            verbose=1, 
                                            save_weights_only=True)

def create_model():

  model = tf.keras.Sequential([
      Dense(128, input_dim=784),
      ReLU(negative_slope=.01),
      Dense(128),
      ReLU(negative_slope=.01),
      Dense(128),
      ReLU(negative_slope=.01),
      Dense(10, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam',
                metrics=['accuracy'])

  return model

model = create_model()

model.fit(X_train, y_train, epochs=2, 
          validation_data=(X_test,y_test),
          verbose=2,
          callbacks=[cpoint])

Epoch 1/2

Epoch 00001: saving model to weights_best.h5
2500/2500 - 9s - loss: 0.6233 - accuracy: 0.8093 - val_loss: 0.5000 - val_accuracy: 0.8474
Epoch 2/2

Epoch 00002: saving model to weights_best.h5
2500/2500 - 9s - loss: 0.4467 - accuracy: 0.8632 - val_loss: 0.4568 - val_accuracy: 0.8610


In [23]:
model2 = create_model()
model2.load_weights('./weights_best.h5') #Loads weights
model2.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 128)               100480    
_________________________________________________________________
re_lu_27 (ReLU)              (None, 128)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 128)               16512     
_________________________________________________________________
re_lu_28 (ReLU)              (None, 128)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 128)               16512     
_________________________________________________________________
re_lu_29 (ReLU)              (None, 128)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 10)               

In [36]:
model2.evaluate(X_test,y_test)

625/625 [==============================] - 1s 2ms/step - loss: 0.4568 - accuracy: 0.8610


[0.4568011164665222, 0.8610000014305115]

In [31]:
# Save entire model
# i.e. both weights and architecture
# Create and train a new model instance.
model = create_model()
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
2500/2500 [==============================] - 10s 4ms/step - loss: 0.6355 - accuracy: 0.8057 - val_loss: 0.5076 - val_accuracy: 0.8432
Epoch 2/5
2500/2500 [==============================] - 10s 4ms/step - loss: 0.4523 - accuracy: 0.8609 - val_loss: 0.4571 - val_accuracy: 0.8581
Epoch 3/5
2500/2500 [==============================] - 10s 4ms/step - loss: 0.3896 - accuracy: 0.8800 - val_loss: 0.4521 - val_accuracy: 0.8649
Epoch 4/5
2500/2500 [==============================] - 10s 4ms/step - loss: 0.3473 - accuracy: 0.8928 - val_loss: 0.4343 - val_accuracy: 0.8710
Epoch 5/5
2500/2500 [==============================] - 10s 4ms/step - loss: 0.3117 - accuracy: 0.9031 - val_loss: 0.4353 - val_accuracy: 0.8724
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [32]:
model.evaluate(X_test, y_test)

625/625 [==============================] - 1s 2ms/step - loss: 0.4353 - accuracy: 0.8724


[0.4353192448616028, 0.8724499940872192]

In [37]:
# Load it back fresh, including weights and architecture

new_model = tf.keras.models.load_model('saved_model/my_model')
new_model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'])
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 128)               100480    
_________________________________________________________________
re_lu_33 (ReLU)              (None, 128)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 128)               16512     
_________________________________________________________________
re_lu_34 (ReLU)              (None, 128)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 128)               16512     
_________________________________________________________________
re_lu_35 (ReLU)              (None, 128)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 10)              

In [39]:
new_model.evaluate(X_test,y_test)

625/625 [==============================] - 1s 2ms/step - loss: 0.4353 - accuracy: 0.8724


[0.4353192448616028, 0.8724499940872192]

In [28]:
!ls -alh  # Half a meg for a model

total 25M
drwxr-xr-x 1 root root 4.0K Sep 10 21:45 .
drwxr-xr-x 1 root root 4.0K Sep 10 21:24 ..
drwxr-xr-x 1 root root 4.0K Aug 31 16:13 .config
drwxr-xr-x 6 root root 4.0K Sep 10 21:33 logs
-rw-r--r-- 1 root root  25M Sep 10 21:28 quickdraw10.npz
drwxr-xr-x 1 root root 4.0K Aug 27 16:39 sample_data
drwxr-xr-x 3 root root 4.0K Sep 10 21:45 saved_model
-rw-r--r-- 1 root root 546K Sep 10 21:41 weights_best.h5


### Stretch Goals
- Mount your Google Drive to Colab to persist your model checkpoint files. 
- Research L2 normalization (weight decay)
- Write a custom callback function to stop training after you reach .88 validation accuracy. 
- Select a new dataset and apply a neural network to it.
- Research TensorFlow Serving
- Play [QuickDraw](https://quickdraw.withgoogle.com/data)
- Create a static webpage using TensorFlow.js to serve a model. Check out [Teachable Machine Learning](https://teachablemachine.withgoogle.com/) for ideas. 